In [ ]:
from tabulate import tabulate
import random

class Tambola():
    def __init__(self):
        # Define the no. of tickets
        self.no_of_tickets = 6
        self.tickets_per_player = [6]  # one player => 6 tickets in serial ticket generation
        self.player_tickets = []

    def start_game(self):
        self.players_summary = {i:{'no_of_tickets':no_of_tickets} for i,no_of_tickets in enumerate(self.tickets_per_player)}
        for i, tickets in enumerate(self.player_tickets):
            row_numbers = [sum(1 for item in row if isinstance(item,(int))) for row in tickets]
            ticket_numbers = [sum(num for num in row_numbers[t_no*3:(t_no+1)*3]) for t_no in range(self.tickets_per_player[i])]
            self.players_summary[i]['numbers_in_row'] = row_numbers
            self.players_summary[i]['numbers_in_ticket'] = ticket_numbers
            self.players_summary[i]['numbers_marked_in_ticket'] = [0 for _ in range(self.tickets_per_player[i])]
            self.players_summary[i]['tickets'] = tickets
        return self.players_summary

    def reset_generate_tickets(self):
        self.ticket_array = [[None for _ in range(9)] for _ in range(3*6)]

        # a serial ==> 6 tickets with all 1-90 numbers
        self.total_numbers = {i:[j for j in range(i*10,10*(i+1))] for i in range(9)}
        self.total_numbers[0].remove(0)
        self.total_numbers[8].append(90)

        # Define the ranges for each column
        # maintain column_ranges to fill row columns according to conditions
        self.column_ranges = {i: len(self.total_numbers[i])  for i in range(9)}

        # total_rows ==> 6*3 (each ticket 3 rows)
        self.total_rows = 18

    def get_3rd_row(self,first_row, second_row):
        third_row = [i for i in range(9)]
        common_columns = set(first_row) & set(second_row)

        fixed_row_choices = []

        remaining_rows = self.total_rows
        self.total_rows -= 1
        for i in range(9):
            if(self.column_ranges[i]==remaining_rows):
                fixed_row_choices.append(i)
                self.column_ranges[i] -= 1
                third_row.remove(i)
            elif(self.column_ranges[i]<=0 or i in common_columns):
                third_row.remove(i)
        # worst case ==> first_row & second_row all 5 chosen are same. ==> len(third_row) = 4
        if(len(third_row)==4 or len(common_columns)==5):
            extra_choice = random.sample(list(common_columns), 1)
            third_row = third_row + extra_choice

        third_row = random.sample(third_row, 5-len(fixed_row_choices))
        for i in third_row:
            self.column_ranges[i] -= 1
        return third_row + fixed_row_choices


    def get_row_indices(self):
        # Define the row incides
        total_row_indices = [i for i in range(9)]
        fixed_row_choices = []
        remaining_rows = self.total_rows
        self.total_rows -= 1
        for i in range(9):
            if(self.column_ranges[i]==remaining_rows):
                fixed_row_choices.append(i)
                self.column_ranges[i] -= 1
                total_row_indices.remove(i)
            elif(self.column_ranges[i]<=0):
                total_row_indices.remove(i)
        row_indices = random.sample(total_row_indices, 5-len(fixed_row_choices))
        for i in row_indices:
            self.column_ranges[i] -= 1
        return row_indices + fixed_row_choices

    def get_row_choices(self):
        first_row = self.get_row_indices()
        second_row = self.get_row_indices()
        third_row = self.get_3rd_row(first_row, second_row)
        return [first_row, second_row, third_row]

    def get_random_ticket(self,ticket_no):
        row_choices = self.get_row_choices()
        for i,row_choice in enumerate(row_choices):
            for column in row_choice:
                number = random.choice(self.total_numbers[column])
                self.ticket_array[i+(ticket_no*3)][column] = number
                self.total_numbers[column].remove(number)

    # only serial of 6 tickets ==> all 1-90 numbers
    def genarate_serial_tickets(self):
        self.reset_generate_tickets()
        for ticket_no in range(6):
            self.get_random_ticket(ticket_no)
            random_ticket = self.ticket_array[ticket_no*3:3*(ticket_no+1)]
            print(tabulate(random_ticket, tablefmt="fancy_grid", numalign="center"))

    # random serials per each player with choosen no_of_tickets ==> example [2,3,4]
    # player 0 ==> 2 tickets
    # player 1 ==> 3 tickets
    # player 2 ==> 4 tickets
    def genarate_tickets_per_player(self, tickets_per_player):
        self.tickets_per_player = tickets_per_player
        for tickets in self.tickets_per_player:
            self.reset_generate_tickets()
            if(tickets>6):
                continue
            self.ticket_array = [[None for _ in range(9)] for _ in range(3*6)]
            for ticket_no in range(tickets):
                self.get_random_ticket(ticket_no)

            self.player_tickets.append(self.ticket_array[:tickets*3])

    def get_tickets(self):
        for i, tickets in enumerate(self.player_tickets):
            print(f'player {i} tickets are {self.tickets_per_player[i]}')
            for ticket_no in range(self.tickets_per_player[i]):
                random_ticket = tickets[ticket_no*3:3*(ticket_no+1)]
                print(tabulate(random_ticket, tablefmt="heavy_grid", numalign="center"))
            print('\n\n____________________________________________________________\n\n')



# Example usage:
tambola_game = Tambola()
tambola_game.reset_generate_tickets()

# generate tickets
#tambola_game.genarate_serial_tickets()
tambola_game.genarate_tickets_per_player([2,3,4])
tambola_game.get_tickets()
players_summary = tambola_game.start_game()

player 0 tickets are 2
┏━━━┳━━━━┳━━━━┳━━━━┳━━━━┳━━━━┳━━━━┳━━━━┳━━━━┓
┃ 2 ┃ 18 ┃ 23 ┃    ┃ 49 ┃    ┃    ┃ 74 ┃    ┃
┣━━━╋━━━━╋━━━━╋━━━━╋━━━━╋━━━━╋━━━━╋━━━━╋━━━━┫
┃ 4 ┃    ┃ 27 ┃ 36 ┃ 43 ┃    ┃    ┃    ┃ 90 ┃
┣━━━╋━━━━╋━━━━╋━━━━╋━━━━╋━━━━╋━━━━╋━━━━╋━━━━┫
┃   ┃ 13 ┃    ┃    ┃    ┃ 59 ┃ 61 ┃ 76 ┃ 84 ┃
┗━━━┻━━━━┻━━━━┻━━━━┻━━━━┻━━━━┻━━━━┻━━━━┻━━━━┛
┏━━━┳━━━━┳━━━━┳━━━━┳━━━━┳━━━━┳━━━━┳━━━━┳━━━━┓
┃ 7 ┃ 17 ┃ 29 ┃    ┃ 45 ┃    ┃    ┃    ┃ 89 ┃
┣━━━╋━━━━╋━━━━╋━━━━╋━━━━╋━━━━╋━━━━╋━━━━╋━━━━┫
┃ 5 ┃    ┃ 21 ┃ 32 ┃    ┃    ┃ 69 ┃    ┃ 86 ┃
┣━━━╋━━━━╋━━━━╋━━━━╋━━━━╋━━━━╋━━━━╋━━━━╋━━━━┫
┃   ┃ 10 ┃    ┃ 37 ┃ 48 ┃ 56 ┃    ┃ 77 ┃    ┃
┗━━━┻━━━━┻━━━━┻━━━━┻━━━━┻━━━━┻━━━━┻━━━━┻━━━━┛


____________________________________________________________


player 1 tickets are 3
┏━━━┳━━━━┳━━━━┳━━━━┳━━━━┳━━━━┳━━━━┳━━━━┳━━━━┓
┃ 6 ┃    ┃ 24 ┃    ┃ 45 ┃ 57 ┃ 61 ┃    ┃    ┃
┣━━━╋━━━━╋━━━━╋━━━━╋━━━━╋━━━━╋━━━━╋━━━━╋━━━━┫
┃ 2 ┃    ┃    ┃ 37 ┃    ┃ 59 ┃ 66 ┃ 77 ┃    ┃
┣━━━╋━━━━╋━━━━╋━━━━╋━━━━╋━━━━╋━━━━╋━━━━╋━━━━┫
┃   ┃ 19 ┃ 21 ┃

In [ ]:
players_summary

{0: {'no_of_tickets': 2,
  'numbers_in_row': [5, 5, 5, 5, 5, 5],
  'numbers_in_ticket': [15, 15],
  'numbers_marked_in_ticket': [0, 0],
  'tickets': [[7, 16, None, None, 45, 59, 67, None, None],
   [None, None, 20, 30, 48, None, 60, 78, None],
   [None, 14, None, 36, None, 55, None, 70, 84],
   [None, 12, None, 33, None, 51, 65, None, 82],
   [4, None, None, 34, 46, 58, 69, None, None],
   [5, 17, 26, None, None, None, None, 75, 87]]},
 1: {'no_of_tickets': 3,
  'numbers_in_row': [5, 5, 5, 5, 5, 5, 5, 5, 5],
  'numbers_in_ticket': [15, 15, 15],
  'numbers_marked_in_ticket': [0, 0, 0],
  'tickets': [[8, None, None, None, 40, None, 60, 73, 87],
   [6, 10, 26, None, 46, None, 66, None, None],
   [None, 12, None, 38, None, 52, None, 77, 82],
   [None, 11, 24, None, 42, 53, 69, None, None],
   [7, None, 25, None, None, 54, 61, None, 90],
   [3, 17, None, 30, 45, None, None, None, 88],
   [1, 13, 27, 33, None, 57, None, None, None],
   [None, 19, 20, None, 41, None, 65, None, 89],
   [2, Non

In [ ]:

class Play_Tambola(object):
    def __init__(self, players_summary):
        self.total_numbers = [i for i in range(1,91)]
        self.players_summary = players_summary
        self.jaldi_game = False
        self.upper_row_game = False
        self.middle_row_game = False
        self.lower_row_game = False
        self.housie_game = False

    def auto_play(self):
        for _ in range(90):
            if(self.housie_game):
                break
            self.generate_random_number()


    def generate_random_number(self):
        random_number = random.choice(self.total_numbers)
        print(f'\nrandom number generated {random_number}')
        self.total_numbers.remove(random_number)

        for player_id, player_data in self.players_summary.items():
            flattened_tickets = [i for row in player_data['tickets'] for i in row]
            if(random_number in flattened_tickets):
                t_index = flattened_tickets.index(random_number)
                row = t_index//9
                col = t_index%9
                t_no = int(row/3)
                print(f'player: {player_id} has generated no. at ({row}, {col}), in ticket_no. {t_no+1}')
                self.players_summary[player_id]['tickets'][row][col] = 'X'
                self.players_summary[player_id]['numbers_in_row'][row]-=1
                self.players_summary[player_id]['numbers_marked_in_ticket'][t_no]+=1
                self.players_summary[player_id]['numbers_in_ticket'][t_no]-=1

                if(self.players_summary[player_id]['numbers_in_row'][row] == 0):
                    if(not self.upper_row_game and row%3==0):
                        print('\n\n$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$\n\n')
                        print(f'player {player_id} won upper row... at {row}')
                        print('\n\n$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$\n\n')
                        self.upper_row_game = True
                    elif(not self.middle_row_game and row%3==1):
                        print('\n\n$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$\n\n')
                        print(f'player {player_id} won middle row... at {row}')
                        print('\n\n$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$\n\n')
                        self.middle_row_game = True
                    elif(not self.lower_row_game and row%3==2):
                        print('\n\n$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$\n\n')
                        print(f'player {player_id} won lower row... at {row}')
                        print('\n\n$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$\n\n')
                        self.lower_row_game = True
                if(self.players_summary[player_id]['numbers_marked_in_ticket'][t_no] == 5 and not self.jaldi_game):
                    print('\n\n$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$\n\n')
                    print(f'player {player_id} won jaldi game.')
                    print('\n\n$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$\n\n')
                    self.jaldi_game = True
                if(self.players_summary[player_id]['numbers_in_ticket'][t_no] == 0 and not self.housie_game):
                    print('\n\n$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$\n\n')
                    print(f'player {player_id} won Tambola game.')
                    print('\n\n$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$\n\n')
                    self.housie_game = True
                    break

play_tambola = Play_Tambola(players_summary)

In [ ]:
play_tambola.auto_play()


random number generated 70
player: 0 has generated no. at (2, 7), in ticket_no. 1
player: 2 has generated no. at (6, 7), in ticket_no. 3

random number generated 14
player: 0 has generated no. at (2, 1), in ticket_no. 1
player: 2 has generated no. at (1, 1), in ticket_no. 1

random number generated 28

random number generated 8
player: 1 has generated no. at (0, 0), in ticket_no. 1
player: 2 has generated no. at (8, 0), in ticket_no. 3

random number generated 24
player: 1 has generated no. at (3, 2), in ticket_no. 2

random number generated 82
player: 0 has generated no. at (3, 8), in ticket_no. 2
player: 1 has generated no. at (2, 8), in ticket_no. 1
player: 2 has generated no. at (1, 8), in ticket_no. 1

random number generated 25
player: 1 has generated no. at (4, 2), in ticket_no. 2
player: 2 has generated no. at (11, 2), in ticket_no. 4

random number generated 26
player: 0 has generated no. at (5, 2), in ticket_no. 2
player: 1 has generated no. at (1, 2), in ticket_no. 1
player

In [ ]:
tambola_game.get_tickets()

player 0 tickets are 2
┏━━━┳━━━┳━━━┳━━━━┳━━━━┳━━━┳━━━┳━━━┳━━━┓
┃ X ┃ X ┃   ┃    ┃ X  ┃ X ┃ X ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━━╋━━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃ X ┃ 30 ┃ 48 ┃   ┃ X ┃ X ┃   ┃
┣━━━╋━━━╋━━━╋━━━━╋━━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃ X ┃   ┃ X  ┃    ┃ X ┃   ┃ X ┃ X ┃
┗━━━┻━━━┻━━━┻━━━━┻━━━━┻━━━┻━━━┻━━━┻━━━┛
┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━━┳━━━┓
┃   ┃ X ┃   ┃ X ┃   ┃ X ┃ X ┃    ┃ X ┃
┣━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━━╋━━━┫
┃ 4 ┃   ┃   ┃ X ┃ X ┃ X ┃ X ┃    ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━━╋━━━┫
┃ 5 ┃ X ┃ X ┃   ┃   ┃   ┃   ┃ 75 ┃ X ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┻━━━┻━━━┻━━━━┻━━━┛


____________________________________________________________


player 1 tickets are 3
┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃ X ┃   ┃   ┃   ┃ X ┃   ┃ X ┃ X ┃ X ┃
┣━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━┫
┃ X ┃ X ┃ X ┃   ┃ X ┃   ┃ X ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃ X ┃   ┃ X ┃   ┃ X ┃   ┃ X ┃ X ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┻━━━┻━━━┻━━━┻━━━┛
┏━━━┳━━━┳━━━┳━━━━┳━━━┳━━━┳━━━┳━━┳━━━━┓
┃   ┃ X ┃ X ┃    ┃ X ┃ X ┃ X ┃ 

In [ ]:

play_tambola.generate_random_number()
play_tambola.players_summary


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90]

random number generated 65


{0: {'no_of_tickets': 2,
  'numbers_in_row': [5, 5, 5, 5, 5, 5],
  'numbers_in_ticket': [15, 15],
  'numbers_marked_in_ticket': [0, 0],
  'tickets': [[6, 10, None, 35, None, None, 69, None, 87],
   [1, None, None, 36, None, 57, None, 70, 85],
   [None, 13, 21, None, 42, 55, 63, None, None],
   [None, 18, 20, None, 46, None, 68, None, 86],
   [None, 11, None, None, 41, 59, None, 77, 80],
   [8, None, 25, None, None, 50, 64, 74, None]]},
 1: {'no_of_tickets': 3,
  'numbers_in_row': [5, 5, 5, 5, 5, 5, 5, 5, 5],
  'numbers_in_ticket': [15, 15, 15],
  'numbers_marked_in_ticket': [0, 0, 0],
  'tickets': [[5, 10, 26, None, None, None, None, 71, 80],
   [None, 16, None, 37, None, 51, None, 75, 88],
   [8, None, None, 33, 42, 53, 62, None, None],
   [3, 13, None, None, 44, 52, None, None, 83],
   [None, 19, 23, 32, None, 59, 64, None, None],
   [2, None, None, 31, 45, None, 61, None, 87],
   [None, None, 24, 35, 43, None, 60, None, 89],
   [None, 15, 28, None, None, 55, None, 78, 84],
   [9, No

In [ ]:
from tabulate import tabulate

# Define the no. of tickets
no_of_tickets = 6

ticket_array = [[None for _ in range(9)] for _ in range(3*6)]


total_numbers = {i:[j for j in range(i*10,10*(i+1))] for i in range(9)}
total_numbers[0].remove(0)
total_numbers[8].append(90)
#print(total_numbers)

# Define the ranges for each column
column_ranges = {i: len(total_numbers[i])  for i in range(9)}


# total_rows
total_rows = 18
fixed_row_choices = []

def get_3rd_row(first_row, second_row):
    common_columns = set(first_row) & set(second_row)
    third_row = [i for i in range(9)]
    global total_rows, fixed_row_choices
    fixed_row_choices = []
    remaining_rows = total_rows
    total_rows -= 1
    for i in range(9):
        if(column_ranges[i]==remaining_rows):
            fixed_row_choices.append(i)
            column_ranges[i] -= 1
        if(column_ranges[i]<=0 or i in common_columns):
            third_row.remove(i)
    third_row = random.sample(third_row, 5-len(fixed_row_choices))
    for i in third_row:
        column_ranges[i] -= 1
    return third_row + fixed_row_choices


def get_row_indices():
    # Define the row incides
    total_row_indices = [i for i in range(9)]
    global total_rows, fixed_row_choices
    fixed_row_choices = []
    remaining_rows = total_rows
    total_rows -= 1
    for i in range(9):
        if(column_ranges[i]==remaining_rows):
            fixed_row_choices.append(i)
            column_ranges[i] -= 1
        if(column_ranges[i]<=0):
            total_row_indices.remove(i)
    row_indices = random.sample(total_row_indices, 5-len(fixed_row_choices))
    for i in row_indices:
        column_ranges[i] -= 1
    return row_indices + fixed_row_choices

def get_row_choices():
    first_row = get_row_indices()
    second_row = get_row_indices()
    third_row = get_3rd_row(first_row, second_row)
    return [first_row, second_row, third_row]

def get_random_ticket(ticket_no,row_choices):
    for i,row_choice in enumerate(row_choices):
        for column in row_choice:
            number = random.choice(total_numbers[column])
            ticket_array[i+(ticket_no*3)][column] = number
            total_numbers[column].remove(number)
    return ticket_array[ticket_no*3:3*(ticket_no+1)]


for ticket_no in range(no_of_tickets):
    row_choices = get_row_choices()

    # get random_no's b/w 0-90 for one ticket
    random_ticket = get_random_ticket(ticket_no,row_choices)
    print(tabulate(random_ticket, tablefmt="fancy_grid", numalign="center"))

╒═══╤════╤════╤════╤════╤════╤════╤════╤════╕
│ 1 │    │ 26 │    │ 44 │    │ 68 │ 73 │    │
├───┼────┼────┼────┼────┼────┼────┼────┼────┤
│ 6 │    │    │ 34 │ 41 │    │ 65 │    │ 90 │
├───┼────┼────┼────┼────┼────┼────┼────┼────┤
│   │ 10 │ 23 │ 36 │    │ 52 │    │ 79 │    │
╘═══╧════╧════╧════╧════╧════╧════╧════╧════╛
╒═══╤════╤════╤════╤════╤════╤════╤════╤════╕
│   │ 16 │ 25 │ 37 │    │ 56 │    │ 76 │    │
├───┼────┼────┼────┼────┼────┼────┼────┼────┤
│   │ 12 │ 29 │ 38 │ 48 │ 54 │    │    │    │
├───┼────┼────┼────┼────┼────┼────┼────┼────┤
│ 5 │    │    │    │ 49 │    │ 69 │ 78 │ 84 │
╘═══╧════╧════╧════╧════╧════╧════╧════╧════╛
╒═══╤══╤════╤════╤════╤════╤════╤════╤════╕
│ 2 │  │ 24 │ 33 │ 43 │ 51 │    │    │    │
├───┼──┼────┼────┼────┼────┼────┼────┼────┤
│ 8 │  │    │    │ 47 │ 50 │    │ 77 │ 82 │
├───┼──┼────┼────┼────┼────┼────┼────┼────┤
│   │  │ 28 │ 32 │    │    │ 67 │ 70 │ 80 │
╘═══╧══╧════╧════╧════╧════╧════╧════╧════╛
╒═══╤════╤════╤══╤════╤════╤════╤════╤════╕
│   